This tutorial is automatically generated from the file test/python/cell_based/tutorials/TestCellSortingTutorial.py.



In [1]:
# Jupyter notebook specific imports 
import matplotlib as mpl 
from IPython import display 
%matplotlib inline


# Introduction
This test is a demonstration of cell sorting using a Cellular Potts based framework.
It shows:
 * How to set up a Potts simulation
 * Working with labels
 
## The Test



In [2]:
import matplotlib.pyplot as plt # Plotting
import numpy as np # Matrix tools
import chaste # The PyChaste module
chaste.init() # Set up MPI
import chaste.cell_based # Contains cell populations
import chaste.mesh # Contains meshes
import chaste.visualization # Visualization tools

## Test 1 - Cell sorting
The next test generates a collection of cells, there are two types of cells, labelled ones and non labelled ones,
there is differential adhesion between the cell types. For the parameters specified, the cells sort into separate types.



In [3]:
# Set up the test 
chaste.cell_based.SetupNotebookTest()

First, we generate a `Potts` mesh. To create a `PottsMesh`, we can use the `PottsMeshGenerator`.
This generates a regular square-shaped mesh, in which all elements are the same size.
We have chosen an 8 by 8 block of elements each consisting of 4 by 4 ( = 16) lattice sites.



In [4]:
generator = chaste.mesh.PottsMeshGenerator2(50, 8, 4, 50, 8, 4)
mesh = generator.GetMesh()

Having created a mesh, we now create some cells. To do this, we the `CellsGenerator` helper class,
as before but this time the third argument is set to make all cells non-proliferative.



In [5]:
differentiated_type = chaste.cell_based.DifferentiatedCellProliferativeType()
cell_generator = chaste.cell_based.CellsGeneratorUniformCellCycleModel_2()
cells = cell_generator.GenerateBasicRandom(mesh.GetNumElements(), differentiated_type)

Before we make a CellPopulation we make a cell label and then assign this label to some randomly chosen cells.



In [6]:
label = chaste.cell_based.CellLabel()
for eachCell in cells:
    if(chaste.core.RandomNumberGenerator.Instance().ranf()<0.5):
        eachCell.AddCellProperty(label)

Now we have a mesh and a set of cells to go with it, we can create a `CellPopulation`.



In [7]:
cell_population = chaste.cell_based.PottsBasedCellPopulation2(mesh, cells)

In order to visualize labelled cells we need to use the following command.



In [8]:
cell_population.AddCellWriterCellLabelWriter()

PyChaste can do simple 3D rendering with VTK. We set up a VtkScene so that we can
see the population evovle in real time.



In [10]:
scene= chaste.visualization.VtkScene2()
scene.SetCellPopulation(cell_population)
scene.GetCellPopulationActorGenerator().SetShowPottsMeshEdges(True)
nb_manager = chaste.visualization.JupyterNotebookManager()
#nb_manager.vtk_show(scene, height=600)

We then pass in the cell population into an `OffLatticeSimulation`, and set the output directory and end time



In [11]:
simulator = chaste.cell_based.OnLatticeSimulation2(cell_population)
simulator.SetOutputDirectory("Python/TestCellSorting")
simulator.SetEndTime(20.0)
simulator.SetSamplingTimestepMultiple(10)

We must now create one or more update rules, which determine the Hamiltonian in the Potts simulation.
For this test, we use two update rules based upon a volume constraint (`VolumeConstraintPottsUpdateRule`) and
differential adhesion between cells (`DifferentialAdhesionPottsUpdateRule`), set appropriate parameters, and
pass them to the `OnLatticeSimulation`.



In [12]:
volume_constraint_update_rule = chaste.cell_based.VolumeConstraintPottsUpdateRule2()
volume_constraint_update_rule.SetMatureCellTargetVolume(16)
volume_constraint_update_rule.SetDeformationEnergyParameter(0.2)
simulator.AddUpdateRule(volume_constraint_update_rule)

We repeat the process for any other update rules.



In [13]:
differential_adhesion_update_rule = chaste.cell_based.DifferentialAdhesionPottsUpdateRule2()
differential_adhesion_update_rule.SetLabelledCellLabelledCellAdhesionEnergyParameter(0.16)
differential_adhesion_update_rule.SetLabelledCellCellAdhesionEnergyParameter(0.11)
differential_adhesion_update_rule.SetCellCellAdhesionEnergyParameter(0.02)
differential_adhesion_update_rule.SetLabelledCellBoundaryAdhesionEnergyParameter(0.16)
differential_adhesion_update_rule.SetCellBoundaryAdhesionEnergyParameter(0.16)
simulator.AddUpdateRule(differential_adhesion_update_rule)

Set up plotting



In [14]:
scene_modifier = chaste.visualization.JupyterSceneModifier2(nb_manager)
scene_modifier.SetVtkScene(scene)
scene_modifier.SetUpdateFrequency(1000)
simulator.AddSimulationModifier(scene_modifier)

To run the simulation, we call `Solve()`.



In [15]:
scene.Start()
simulator.Solve()
# Tear down the test 
chaste.cell_based.TearDownNotebookTest()

TypeError: Unable to convert function return value to a Python type! The signature was
	(self: chaste.visualization._chaste_project_PyChaste_visualization.VtkScene2) -> vtkSmartPointer<vtkRenderer>